# Prepare Dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Model

In [ ]:
train_dir = '../../dataset/trainingSample/trainingSample/'
all_image_paths = []
all_image_labels = []
for label in os.listdir(train_dir):
    for image_path in os.listdir(train_dir + label + '/'):
        all_image_paths.append(train_dir + label + '/' + image_path)
        all_image_labels.append(int(label))

In [ ]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)

In [ ]:
def preprocess_image(image):
    image = tf.image.decode_image(image)
    image = tf.cast(image, tf.float32) / 255.0
    return image
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

In [ ]:
image = load_and_preprocess_image(all_image_paths[0])
plt.imshow(image, cmap='gray')

In [ ]:
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))

In [ ]:
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

In [ ]:
for (image, label) in image_label_ds.take(2):
    plt.imshow(image, cmap='gray')
    plt.xlabel(str(label))
    plt.show()

In [ ]:
BATCH_SIZE = 16
image_label_ds = image_label_ds.shuffle(1000)
image_label_ds = image_label_ds.batch(batch_size=BATCH_SIZE, 
                                      drop_remainder=True)
    
image_label_ds = image_label_ds.repeat()
image_label_ds = image_label_ds.prefetch(tf.data.experimental.AUTOTUNE)

# Create model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Model

In [ ]:
def create_model():
    input_image = Input(shape=(28, 28, 1))
    
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_image)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)

    x = layers.Flatten()(x)
    x = layers.Dense(10)(x)
    
    model = Model(input_image, x)
    
    return model

In [ ]:
model = create_model()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

# Training

In [ ]:
model.fit(x=image_label_ds,
          steps_per_epoch=len(all_image_paths) // BATCH_SIZE,
          epochs=100,
          verbose=1)

In [ ]:
model.save('classification')